<a href="https://colab.research.google.com/github/jasonjstewart/college-football/blob/master/EDA_College_Football_4th_Downs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import pandas as pd
df=pd.read_csv('https://jasonstewart.s3.amazonaws.com/4th-down-data.csv')
#THIS IS AWESOME WOOHOO
df.drop(['yard_line.1','yards_to_goal.1','play_text','offense.1','offense_conference.1','defense.1','defense_conference.1','id.2','start_date','home_id','home_team','home_points','away_team','away_conference','away_points','home_conference'],axis=1, inplace=True)
df.describe()

,id,offense_score,defense_score,drive_id,period,clock,yard_line,yards_to_goal,down,distance,yards_gained,ppa,id.1,game_id,start_period,start_yardline,start_yards_to_goal,end_period,end_yardline,end_yards_to_goal,plays,yards,season,week,attendance,home_post_win_prob,away_id,away_post_win_prob,excitement_index,success_or_failure
count,9.363000e+03,9363.000000,9363.000000,9.363000e+03,9363.000000,9363.000000,9363.000000,9363.000000,9363.0,9363.000000,9363.000000,9363.000000,9.363000e+03,9.363000e+03,9363.000000,9363.000000,9363.000000,9363.000000,9363.000000,9363.000000,9363.000000,9363.000000,9363.000000,9363.000000,9363.000000,9363.000000,9363.000000,9363.000000,9363.000000,9363.000000
mean,1.748109e+17,16.135747,18.451244,3.030587e+10,2.958133,399.274164,52.397949,34.272349,4.0,4.776781,3.991990,-0.231412,3.030587e+10,3.422088e+08,2.860942,48.578874,67.052227,2.978746,54.357898,22.424116,8.886575,45.420592,2012.517142,7.484460,37779.529638,0.454381,716.016448,0.251162,2.561558,0.386521
std,1.988009e+17,13.754774,14.768275,1.045888e+10,1.132420,272.574289,25.875182,20.685511,0.0,4.836324,9.456556,2.180676,1.045888e+10,5.421729e+07,1.151657,24.431232,17.552648,1.126897,36.012999,23.567325,3.558549,26.505455,4.080031,3.944803,34076.951727,0.428832,994.524463,0.352209,2.485868,0.486978
min,2.624300e+11,0.000000,0.000000,4.005477e+09,1.000000,0.000000,0.000000,0.000000,4.0,-20.000000,-83.000000,-10.023488,4.005477e+09,2.624300e+08,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,-39.000000,2006.000000,1.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000
25%,2.928300e+11,7.000000,7.000000,2.824126e+10,2.000000,152.000000,33.000000,20.000000,4.0,1.000000,0.000000,-1.942269,2.824126e+10,2.928301e+08,2.000000,25.000000,59.000000,2.000000,23.000000,0.000000,6.000000,25.000000,2009.000000,4.000000,0.000000,0.000000,99.000000,0.000000,0.000000,0.000000
50%,3.232900e+11,14.000000,17.000000,3.126702e+10,3.000000,374.000000,54.000000,34.000000,4.0,3.000000,1.000000,0.000000,3.126702e+10,3.232902e+08,3.000000,46.000000,72.000000,3.000000,57.000000,17.000000,9.000000,47.000000,2012.000000,8.000000,41113.000000,0.404200,204.000000,0.016200,2.321189,0.000000
75%,4.008690e+17,24.000000,28.000000,4.007569e+10,4.000000,625.000000,71.000000,46.500000,4.0,7.000000,6.000000,1.490412,4.007569e+10,4.008695e+08,4.000000,72.000000,79.000000,4.000000,91.000000,38.000000,11.000000,68.000000,2016.000000,11.000000,61500.000000,0.950600,344.000000,0.482300,4.541697,1.000000
max,4.011330e+17,80.000000,77.000000,4.011330e+10,9.000000,900.000000,99.000000,100.000000,4.0,53.000000,96.000000,7.295661,4.011330e+10,4.011330e+08,9.000000,99.000000,100.000000,9.000000,100.000000,100.000000,23.000000,105.000000,2019.000000,15.000000,110633.000000,1.000000,2755.000000,0.999400,10.807980,1.000000


In [0]:
df=df[df['drive_result']!='KICKOFF']
df.drive_result.unique()

points_scored=[]
for row in range(len(df)):
  if df['drive_result'].iloc[row] in ['END OF 4TH QUARTER,''POSSESSION (FOR OT DRIVES)','DOWNS','DOWNS TD','FG TD','MISSED FG TD','INT','FUMBLE','TURNOVER ON DOWNS','INT TD','FUMBLE TD','FUMBLE RETURN TD','SF','FG MISSED','END OF HALF','END OF GAME','PUNT','Uncategorized','SF','PUNT RETURN TD','FUMBLE RETURN TD','FG MISSED TD','TURNOVER ON DOWNS TD','INT RETURN TOUCH','PUNT TD','MISSED FG',]:
    points_scored.append(0)
  elif df['drive_result'].iloc[row] in ['TD','PASSING TD','RUSHING TD','END OF HALF TD','END OF GAME TD','POSSESSION (FOR OT DRIVES) TD']:
    points_scored.append(6)
  elif df['drive_result'].iloc[row] in ['FG GOOD','FG GOOD TD','FG']:
    points_scored.append(3)
  else:
    points_scored.append(0)

df['points_scored']=points_scored

In [0]:
#set away or home as 0 or 1 categorical
home=[]
for num in range(len(df)):
  #print(df['offense'].iloc[num],len(df['offense'].iloc[num]),df['home'].iloc[num],len(df['home'].iloc[num]))
  hometeam=df['home'].iloc[num]
  if df['offense'].iloc[num]==hometeam [:len(df['offense'].iloc[num])]:
    home.append(1)
  else:
    home.append(0)
df['home']=home
df.drop('away',axis=1, inplace=True)

In [0]:
# Creates Next Drive columns whih while not used in this specific project could be used in the future

# df_drives=pd.read_csv('https://jasonstewart.s3.amazonaws.com/drives.csv')
# df_drives

# for col in df_drives:
#   name='next_drive_'+col
#   df_drives.rename(columns={col : name}, inplace=True)
# df_drives

# for col in df_drives:
#   df[col]=""

# for num in range(len(df)):
#   drive_id=df['id.1'].iloc[num]+1
#   #print(drive_id,df['id.1'].iloc[num])
#   next_drive=df_drives[df_drives['next_drive_id']==drive_id]
#   #print(next_drive['next_drive_id'].iloc[0],df['id.1'].iloc[num])
#   if not next_drive.empty:
#     for col in next_drive:
#       #print(next_drive['next_drive_id'].iloc[0],df['id.1'].iloc[num])
#       df[col].iloc[num]=next_drive[col].iloc[0]
# print(df.columns)
# print(df_drives.columns)

# #df_drives.drop(['offense','offense_conference','defense','defense_conference' ] ,axis=1)
# df['drive_id']=df['id.1']
# df.drop(['id.1'],axis=1,inplace=True)
# df['drive_id']


In [0]:
#FIX SKEWNESS

def fix_skew(df, feature):
  import numpy as np
  # check for positive skewness
  if df[feature].skew() > 1:
    # generate the new columns to test
    df[feature + '_SR'] = np.sqrt(df[feature])
    df[feature + '_CR'] = np.cbrt(df[feature])
    df[feature + '_LN'] = np.log(df[feature] + 1)
    # df[feature + '_BIN'] = bin_numeric(df[feature])[2]
    print(df.skew())
    
    # get dummies
    for col in df:
      if not pd.api.types.is_numeric_dtype(df[col]):
        df = df.join(pd.get_dummies(df[col], prefix=col, drop_first=True))

    # get numeric columns
    df = df.select_dtypes(np.number)
    df_minmax = pd.DataFrame(preprocessing.MinMaxScaler().fit_transform(df), columns=df.columns)

    # test original feature
    y = df_minmax[feature]
    X = df_minmax.drop(columns=[feature, feature + '_LN', feature + '_SR', feature + '_CR']).assign(const=1)

    results = sm.OLS(y, X).fit()
    print(results.summary())

    # test square root feature
    y = df_minmax[feature + '_SR']
    X = df_minmax.drop(columns=[feature, feature + '_LN', feature + '_SR', feature + '_CR']).assign(const=1)

    results = sm.OLS(y, X).fit()
    print(results.summary())

    # test cube root feature
    y = df_minmax[feature + '_CR']
    X = df_minmax.drop(columns=[feature, feature + '_LN', feature + '_SR', feature + '_CR']).assign(const=1)

    results = sm.OLS(y, X).fit()
    print(results.summary())

    # test log feature
    y = df_minmax[feature + '_LN']
    X = df_minmax.drop(columns=[feature, feature + '_LN', feature + '_SR', feature + '_CR']).assign(const=1)

    results = sm.OLS(y, X).fit()
    print(results.summary())

    # test binning feature
    # y = df_minmax[feature + '_BIN']
    # X = df_minmax.drop(columns=[feature, feature + '_LN', feature + '_SR', feature + '_CR', feature + '_BIN']).assign(const=1)

    # results = sm.OLS(y, X).fit()
    # print(results.summary())


  elif df[feature].skew() < -1:
    # here we would fix negatively skewed variables if there were any in this dataset

    return
  else:
    # return if skewness is between -1 and 1; negligible
    return

In [19]:
#CORRECT OUTLIERS, this first part checks to see if there are outliers
outliers = pd.DataFrame(columns=['min', 'count below', 'max', 'count above'])

# save the max to compare later
# peak_max = df[''].mean() + (df[''].std() * 3)

# Loop through each column in the dataframe except those that are: 1) categorical, 2) empty, or 3) binary (0/1)
for col in df:
  if pd.api.types.is_numeric_dtype(df[col]) and (len(df[col].value_counts()) > 0) and not all(df[col].value_counts().index.isin([0, 1])):
    # Calculate the theoretical min/max
    min = df[col].mean() - (df[col].std() * 3)
    max = df[col].mean() + (df[col].std() * 3)
    outliers.loc[col] = (min, df[col][df[col] < min].count(), max, df[col][df[col] > max].count())

outliers

,min,count below,max,count above
id,-4.215918e+17,0.0,7.712136e+17,0.0
offense_score,-2.512858e+01,0.0,5.740007e+01,100.0
defense_score,-2.585358e+01,0.0,6.275607e+01,37.0
drive_id,-1.070769e+09,0.0,6.168251e+10,0.0
period,-4.391259e-01,0.0,6.355392e+00,3.0
clock,-4.184487e+02,0.0,1.216997e+03,0.0
yard_line,-2.522760e+01,0.0,1.300235e+02,0.0
yards_to_goal,-2.778418e+01,0.0,9.632888e+01,12.0
down,4.000000e+00,0.0,4.000000e+00,0.0
distance,-9.732191e+00,1.0,1.928575e+01,155.0


In [0]:
#Goes through each column in DF to replace outliers with min or max
def manage_outliers(df):
  for col in df:
    if pd.api.types.is_numeric_dtype(df[col]) and (len(df[col].value_counts()) > 0) and not all(df[col].value_counts().index.isin([0, 1])):
      min = df[col].mean() - (df[col].std() * 3)
      max = df[col].mean() + (df[col].std() * 3)

      # Once the min/max is calculated for a column, take an inner loop through the row values of each column to replace values
      for i, value in enumerate(df[col]): 
        if value < min:      
          df.at[i, col + '_3std'] = min  # replace with min
          df.at[i, 'outlier'] = 1         # identify the record as an outlier for future filtering
        elif value > max:    
          df.at[i, col + '_3std'] = max   # replace with max
          df.at[i, 'outlier'] = 1
        else:                
          df.at[i, col + '_3std'] = value # use original value
          df.at[i, 'outlier'] = 0
    
  # I don't like binary variables like "outlier" becoming floats; cast it back to an int
  # if 'outlier' in df.columns:
  #   df = df.astype({"outlier": int})

  # View the cleaned dataset:
  print(df.head())

In [0]:
#There shouldnt be any missing data, like I am 99% sure there isnt missing data but could be good to double check

#THIS BELOW COMES STRAIGHT FROM THE BOOK BUT IT SHOULD WORK
def manage_missing_data(df):
      import pandas as pd
      from scipy import stats

      for col_y in df:                                        # for each column in df to fill
        if df[col_y].isnull().sum() > 0:                      # if there is missing data to fill
          if is_numeric(df[col_y]):                           # and if that column is numeric
            r_list = df.corr()[col_y].abs()                   # create a correlation matrix for just the col_y feature; get absolute values to eliminate sign (+/-)
            r_list.sort_values(inplace=True, ascending=False) # sort the list descending (we want the second item in the list because the first is itself where r = 1.0)

            col_x = r_list.index[1] # Store the index name of the highest correlation

            # Create a new df with missing values of y and x dropped
            clean_df = df.copy()
            clean_df.dropna(subset=[col_y, col_x], inplace=True)

            # Calculate the regression line: y = m(x) + b
            m, b, r, p, err = stats.linregress(clean_df[col_x], clean_df[col_y])

            # enumerate() allows us to include a count-based index (i) along with the value
            for i, value in enumerate(df[col_y].values):
              if not pd.notnull(value):
                imputed_value = round((m * df[col_x][i]) + b, 1)

                # if the prediction is outside the range of actual values, then replace it with the min/max
                if imputed_value < df[col_y].min():
                  df[col_y][i] = df[col_y].min()
                elif imputed_value > df[col_y].max():
                  df[col_y][i] = df[col_y].max()
                else:
                  df[col_y][i] = imputed_value
      return df

In [0]:
def data_cleaning(df):
  manage_missing_data(df)
  manage_outliers(df)
  for col in df:
    fix_skew(df, col)
  

In [0]:
df.to_csv('college-football-data-p5-4th-down.csv')

In [24]:
df

,offense,id,offense_conference,defense,defense_conference,home,offense_score,defense_score,drive_id,period,clock,yard_line,yards_to_goal,down,distance,yards_gained,play_type,ppa,id.1,game_id,scoring,start_period,start_yardline,start_yards_to_goal,start_time,end_period,end_yardline,end_yards_to_goal,end_time,elapsed,plays,yards,drive_result,season,week,season_type,neutral_site,conference_game,attendance,home_post_win_prob,away_id,away_post_win_prob,excitement_index,success_or_failure,points_scored
0,Georgia,4.006040e+17,SEC,Auburn,SEC,0,0,7,4006039314,1,89,36,36,4,4,5,Pass Reception,2.130077,4006039314,400603931,False,1,66,66,"{'minutes': 4, 'seconds': 32}",2,2,2,"{'minutes': 12, 'seconds': 39}","{'minutes': 6, 'seconds': 53}",12,64,DOWNS,2015,11,regular,False,True,87451,0.6809,61,0.3191,3.706368,0,0
1,Vanderbilt,4.010120e+17,SEC,Tennessee State,None,1,0,0,4010122992,1,696,68,32,4,2,-10,Pass Interception Return,-2.672218,4010122992,401012299,False,1,49,51,"{'minutes': 11, 'seconds': 36}",1,68,32,"{'minutes': 8, 'seconds': 17}","{'minutes': 3, 'seconds': 19}",6,19,INT,2018,5,regular,False,False,0,0.6333,2634,0.3667,6.214617,0,0
2,South Carolina,4.006040e+17,SEC,LSU,SEC,0,3,7,4006039033,1,315,56,56,4,2,0,Pass Incompletion,-0.589364,4006039033,400603903,False,1,75,75,"{'minutes': 5, 'seconds': 15}",1,56,56,"{'minutes': 2, 'seconds': 37}","{'minutes': 2, 'seconds': 38}",5,19,DOWNS,2015,6,regular,False,True,42058,0.9973,2579,0.0027,3.178482,0,0
3,Wake Forest,4.005480e+17,ACC,Army,FBS Independents,1,14,14,4005478328,2,562,65,35,4,11,3,Pass Reception,-1.287230,4005478328,400547832,False,2,38,62,"{'minutes': 9, 'seconds': 22}",2,68,32,"{'minutes': 6, 'seconds': 51}","{'minutes': 2, 'seconds': 31}",8,25,DOWNS,2014,4,regular,False,False,28123,0.5234,349,0.4766,5.788620,0,0
4,Kansas State,4.005480e+17,Big 12,Auburn,SEC,1,0,3,4005478564,1,482,47,53,4,1,6,Penalty,0.000000,4005478564,400547856,False,1,25,75,"{'minutes': 11, 'seconds': 14}",1,80,20,"{'minutes': 5, 'seconds': 21}","{'minutes': 5, 'seconds': 53}",11,74,INT,2014,4,regular,False,False,53046,0.1775,2,0.8225,6.364702,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9358,Florida,4.005480e+17,SEC,Eastern Michigan,Mid-American,1,0,0,4005481012,1,474,76,24,4,2,3,Rush,1.059424,4005481012,400548101,True,1,21,79,"{'minutes': 13, 'seconds': 15}",1,84,16,"{'minutes': 6, 'seconds': 44}","{'minutes': 6, 'seconds': 31}",16,63,FG,2014,2,regular,False,False,81049,1.0000,2199,0.0000,1.374711,1,3
9359,Virginia,4.005480e+17,ACC,Miami,ACC,1,0,7,4005477937,2,900,92,8,4,2,3,Rush,1.093967,4005477937,400547793,True,1,33,67,"{'minutes': 3, 'seconds': 3}",2,97,3,"{'minutes': 11, 'seconds': 16}","{'minutes': 6, 'seconds': 47}",13,64,FG,2014,13,regular,False,True,39112,0.9870,2390,0.0130,3.689861,1,3
9360,Georgia,4.005480e+17,SEC,Auburn,SEC,1,14,7,4005483377,2,528,91,9,4,1,9,Rushing Touchdown,2.943088,4005483377,400548337,True,2,17,83,"{'minutes': 13, 'seconds': 35}",2,100,0,"{'minutes': 8, 'seconds': 48}","{'minutes': 4, 'seconds': 47}",11,83,TD,2014,12,regular,False,True,92746,0.9993,2,0.0007,3.007639,1,6
9361,Missouri,4.005480e+17,SEC,Arkansas,SEC,1,0,7,4005483253,1,73,71,29,4,1,2,Rush,1.203179,4005483253,400548325,True,1,25,75,"{'minutes': 6, 'seconds': 48}",2,66,34,"{'minutes': 14, 'seconds': 23}","{'minutes': 7, 'seconds': 25}",17,41,FG,2014,14,regular,False,True,71168,0.5365,8,0.4635,5.696126,1,3
